<a href="https://colab.research.google.com/github/deep-diver/deeplearning-with-structured-data/blob/master/notebooks/streetcar_data_preparation-geocode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/deep-diver/deeplearning-with-structured-data.git
!mv deeplearning-with-structured-data/* ./

Cloning into 'deeplearning-with-structured-data'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 285 (delta 122), reused 262 (delta 106), pack-reused 0
Receiving objects: 100% (285/285), 21.37 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (122/122), done.


# 경전철 지연 예측 - 지역코드 관련 데이터 준비

미래의 지연을 예측을 예측하고, 지연 발생에 대한 조언을 구하고자 토론토 교통국(TTC)의 경전철 지연 데이터 중 2014년 - 현재까지를 포함한 데이터셋을 사용합니다.

원본 데이터셋의 위치: https://open.toronto.ca/dataset/ttc-streetcar-delay-data/

이 노트북은 자유형식 텍스트로 표현된 위치 정보를 위도/경도에 매핑하는 데이터 준비 과정을 다룹니다.

- 파이썬용 구글 맵스 API 웹 서비스를 사용합니다(https://github.com/googlemaps/google-maps-services-python)
- 위도/경도 값을 얻은 뒤 데이터셋에 해당 데이터를 담을 신규 열을 생성합니다.

# 경전철 경로(routes)

출처: https://www.ttc.ca/Routes/Streetcars.jsp

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/streetcarnov3/master/streetcar%20routes.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# 이전 데이터 준비 단계에서 저장한 데이터프레임을 불러오기

In [1]:
# 선형 대수용 라이브러리
import numpy as np 

# 데이터 전처리용 라이브러리로 CSV 파일 입출력을 위해 불러옵니다(예. pd.read_csv)
import pandas as pd 
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime
import os

remove_bad_values = False
city_name = 'Toronto'


In [4]:
# 노트북이 저장된 디렉터리에 접근합니다
rawpath = os.getcwd()

# Google Colab을 사용하지 않는 경우, 아래의 코드를 주석처리 합니다
rawpath = os.path.join(rawpath, 'notebooks')
print("노트북이 저장된 경로: ",rawpath)

노트북이 저장된 경로:  /content/notebooks


In [9]:
# 데이터는 "data" 디렉터리에 들어있으며, "notebooks" 디렉터리와 같은 계위에 위치합니다
# 데이터 디렉터리에 접근합니다
path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
print("데이터가 저장된 경로: ", path)

데이터가 저장된 경로:  /content/data


In [6]:
# 경전철 문제를 위한 상수
# data_preparation 노트북에서 지정된것과 동일한 값이어야 함: pickled_input_dataframe, pickled_output_dataframe
pickled_data_file = '2014_2018.pkl'
#pickled_dataframe = '2014_2018_df.pkl'
pickled_dataframe = '2014_2018_df_cleaned_keep_bad_apr23.pkl'
pickled_output_dataframe = '2014_2018_df_cleaned_keep_bad_loc_geocoded.pkl'

In [7]:
file_name = os.path.join(path,pickled_dataframe)
df = pd.read_pickle(file_name)
df.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Report Date Time
Report Date Time,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01,505,00:00:00,Friday,dundas west stationt to broadview station,General Delay,7.0,14.0,w,4028,2016-01-01 00:00:00
2016-01-01 02:14:00,2016-01-01,511,02:14:00,Friday,fleet st. and strachan,Mechanical,10.0,20.0,e,4018,2016-01-01 02:14:00
2016-01-01 02:22:00,2016-01-01,301,02:22:00,Friday,queen st. west and roncesvalles,Mechanical,9.0,18.0,w,4201,2016-01-01 02:22:00
2016-01-01 03:28:00,2016-01-01,301,03:28:00,Friday,lake shore blvd. and superior st.,Mechanical,20.0,40.0,e,4251,2016-01-01 03:28:00
2016-01-01 14:28:00,2016-01-01,501,14:28:00,Friday,roncesvalles to neville park,Mechanical,6.0,12.0,e,4242,2016-01-01 14:28:00


In [8]:
df.shape

(69603, 11)

In [14]:
# gapminder['continent'].unique().tolist 만을
# 포함한 데이터프레임을 생성합니다
loc_unique = df['Location'].unique().tolist()
print("Location 고윳값 목록(loc_unique): ", loc_unique[0])

# pd.DataFrame(q_list, columns=['q_data'])
df_unique = pd.DataFrame(loc_unique, columns=['Location'])
df_unique.head()

Location 고윳값 목록(loc_unique):  dundas west stationt to broadview station


,Location
0,dundas west stationt to broadview station
1,fleet st. and strachan
2,queen st. west and roncesvalles
3,lake shore blvd. and superior st.
4,roncesvalles to neville park


In [15]:
df_unique.shape

(10074, 1)

# 지역코드 API 설정

In [16]:
! pip install -U googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.5.3-py3-none-any.whl size=38479 sha256=f77a7cc70f138e5bf3a3e2c08bc6131b2de24b947ba1aecf728f1e6b6e4aba8a
  Stored in directory: /root/.cache/pip/wheels/fa/1a/1c/cc0b8a1652a3f06aea586b0e4714a81bafed830513969baf92
Successfully built googlemaps


In [ ]:
import googlemaps

# API 키는 https://developers.google.com/maps/documentation/embed/get-api-key 에서 얻을 수 있습니다
# NOTE: 아래 코드를 실행하려면 여러분만의 API 키를 생성한 뒤, 아래 `key=''` 부분에 키값을 대입합니다.
gmaps = googlemaps.Client(key='')

# 테스트: 주소에 대한 지역코드를 얻습니다
geocode_result = gmaps.geocode('lake shore blvd. and superior st., Toronto')

print("지역코드 API 호출 결과",geocode_result[0]["geometry"]["location"])

geocode result {'lat': 43.61496169999999, 'lng': -79.4886581}


In [ ]:
# 주어진 주소/교차로에 대해 지역코으 API가 준 위도/경도 목록을 반환합니다

def get_geocode_result(junction):
    geo_string = junction+", "+city_name
    # print("geo_string is", geo_string)
    geocode_result = gmaps.geocode(geo_string)
    # 결과가 비어 있는지 확인합니다
    if len(geocode_result) > 0:
        locs = geocode_result[0]["geometry"]["location"]
        return [locs["lat"], locs["lng"]]
    # 만약 그렇다면 파싱이 불가능 하다는것을 표현하기 위해 0 값들을 반환합니다
    else:
        return [0.0,0.0]

In [ ]:
# 빈 결과를 반환 할 값에 대해 지역코드 API를 테스트 합니다

locs = get_geocode_result("roncesvalles to longbranch")
print("locs ",locs)

locs  [0.0, 0.0]


In [ ]:
# 비어있지 않은 결과를 반환 할 값에 대해 지역코드 API를 테스트 합니다
get_geocode_result("queen and bathurst")[0]

43.6471969

In [ ]:
df.shape

(69603, 11)

In [ ]:
# 지역코드 API가 여러번 호출되는 상황을 피하기 위해서
# 고유한 위치 값만을 가진 데이터프레임에 위도/경도를 담기위한 열을 생성합니다

# 그 후 전체 데이터프레임의 각 데이터의 위치에따라
# 이미 계산된 위도/경도 값을 재사용하여 지역코드 정보를 할당합니다.

df_unique['lat_long'] = df_unique.Location.apply(lambda s: get_geocode_result(s))

In [ ]:
df_unique.head()

,Location,lat_long
0,dundas west stationt to broadview station,"[0.0, 0.0]"
1,fleet st. and strachan,"[43.6362976, -79.4096351]"
2,queen st. west and roncesvalles,"[43.64533489999999, -79.4131843]"
3,lake shore blvd. and superior st.,"[43.61496169999999, -79.4886581]"
4,roncesvalles to neville park,"[0.0, 0.0]"


In [ ]:
df_unique.shape

(10074, 2)

In [ ]:
# 하나의 열에 담긴 위도/경도 정보를 개별 열로 분리합니다
# df["new_col"] = df["A"].str[0]
df_unique["latitude"] = df_unique["lat_long"].str[0]
df_unique["longitude"] = df_unique["lat_long"].str[1]
df_unique.head()

,Location,lat_long,latitude,longitude
0,dundas west stationt to broadview station,"[0.0, 0.0]",0.000000,0.000000
1,fleet st. and strachan,"[43.6362976, -79.4096351]",43.636298,-79.409635
2,queen st. west and roncesvalles,"[43.64533489999999, -79.4131843]",43.645335,-79.413184
3,lake shore blvd. and superior st.,"[43.61496169999999, -79.4886581]",43.614962,-79.488658
4,roncesvalles to neville park,"[0.0, 0.0]",0.000000,0.000000


In [ ]:
df_unique.shape

(10074, 4)

In [ ]:
# 원본 데이터프레임과 df_unique 데이터프레임을 Location 열을 두고 조인시킵니다
# 그러면 원본 데이터프레임에 해당 Location 에 대한 위도/경도 열이 삽입됩니다
# result1 = pd.merge(date_frame, routedirection_frame, on='count', how='outer')
df_out = pd.merge(df, df_unique, on="Location", how='left')
df_out.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Report Date Time,lat_long,latitude,longitude
0,2016-01-01,505,00:00:00,Friday,dundas west stationt to broadview station,General Delay,7.0,14.0,w,4028,2016-01-01 00:00:00,"[0.0, 0.0]",0.000000,0.000000
1,2016-01-01,511,02:14:00,Friday,fleet st. and strachan,Mechanical,10.0,20.0,e,4018,2016-01-01 02:14:00,"[43.6362976, -79.4096351]",43.636298,-79.409635
2,2016-01-01,301,02:22:00,Friday,queen st. west and roncesvalles,Mechanical,9.0,18.0,w,4201,2016-01-01 02:22:00,"[43.64533489999999, -79.4131843]",43.645335,-79.413184
3,2016-01-01,301,03:28:00,Friday,lake shore blvd. and superior st.,Mechanical,20.0,40.0,e,4251,2016-01-01 03:28:00,"[43.61496169999999, -79.4886581]",43.614962,-79.488658
4,2016-01-01,501,14:28:00,Friday,roncesvalles to neville park,Mechanical,6.0,12.0,e,4242,2016-01-01 14:28:00,"[0.0, 0.0]",0.000000,0.000000


In [ ]:
df_out.head(30)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Report Date Time,lat_long,latitude,longitude
0,2016-01-01,505,00:00:00,Friday,dundas west stationt to broadview station,General Delay,7.0,14.0,w,4028,2016-01-01 00:00:00,"[0.0, 0.0]",0.000000,0.000000
1,2016-01-01,511,02:14:00,Friday,fleet st. and strachan,Mechanical,10.0,20.0,e,4018,2016-01-01 02:14:00,"[43.6362976, -79.4096351]",43.636298,-79.409635
2,2016-01-01,301,02:22:00,Friday,queen st. west and roncesvalles,Mechanical,9.0,18.0,w,4201,2016-01-01 02:22:00,"[43.64533489999999, -79.4131843]",43.645335,-79.413184
3,2016-01-01,301,03:28:00,Friday,lake shore blvd. and superior st.,Mechanical,20.0,40.0,e,4251,2016-01-01 03:28:00,"[43.61496169999999, -79.4886581]",43.614962,-79.488658
4,2016-01-01,501,14:28:00,Friday,roncesvalles to neville park,Mechanical,6.0,12.0,e,4242,2016-01-01 14:28:00,"[0.0, 0.0]",0.000000,0.000000
5,2016-01-01,505,15:42:00,Friday,broadview station loop,Investigation,4.0,10.0,w,4187,2016-01-01 15:42:00,"[43.677135, -79.35820799999999]",43.677135,-79.358208
6,2016-01-01,504,15:54:00,Friday,broadview and queen,Mechanical,6.0,12.0,e,4181,2016-01-01 15:54:00,"[43.6593626, -79.34769709999999]",43.659363,-79.347697
7,2016-01-01,501,16:05:00,Friday,roncesvalles to humber loop,Mechanical,6.0,12.0,w,4245,2016-01-01 16:05:00,"[0.0, 0.0]",0.000000,0.000000
8,2016-01-01,506,16:27:00,Friday,main station,Mechanical,8.0,16.0,w,4092,2016-01-01 16:27:00,"[43.6890219, -79.3016857]",43.689022,-79.301686
9,2016-01-01,510,16:34:00,Friday,richmond st. and spadina,Diversion,41.0,46.0,s,bad vehicle,2016-01-01 16:34:00,"[43.6478469, -79.39588049999999]",43.647847,-79.395880


In [ ]:
df_out.shape

(69603, 14)

In [ ]:
print("잘못된 경로의 위도:",df_out[df_out.latitude == 0.0].shape[0])

Bad route latitude: 1675


# 잘못된 행 제거

In [ ]:
print("데이터 정리 후 Location의 수:",df['Location'].nunique())
print("데이터 정리 후 Route의 수:",df['Route'].nunique())
print("데이터 정리 후 Direction의 수:",df['Direction'].nunique())
print("데이터 정리 후 Vehicle의 수:",df['Vehicle'].nunique())
# print("Bad Location count":df[df.Vehicle == 'bad vehicle'].shape[0])
print("잘못된 Route값(bad route)을 가진 데이터 수:",df[df.Route == 'bad route'].shape[0])
print("잘못된 Direction값(bad direction)을 가진 데이터 수:",df[df.Direction == 'bad direction'].shape[0])
print("잘못된 Vehicle값(bad vehicle)을 가진 데이터 수:",df[df.Vehicle == 'bad vehicle'].shape[0])

Location count post cleanup: 10074
Route count post cleanup: 15
Direction count post cleanup: 6
Vehicle count post cleanup: 1017
Bad route count: 3091
Bad direction count: 334
Bad vehicle count: 14480


In [ ]:
# 잘못된 값(bad xxx)을 가진 행들을 제거합니다
if remove_bad_values:
    df = df[df.Vehicle != 'bad vehicle']
    df = df[df.Direction != 'bad direction']
    df = df[df.Route != 'bad route']

In [ ]:
df.shape

(66095, 11)

In [ ]:
pickled_output_dataframe

'2014_2018_df_cleaned_keep_bad_loc_geocoded_apr23.pkl'

In [ ]:
# 정리된 데이터프레임을 피클로 저장합니다
file_name = path + pickled_output_dataframe
df_out.to_pickle(file_name)

In [ ]:
dfn = pd.read_pickle(file_name)
dfn.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Report Date Time,lat_long,latitude,longitude
0,2016-01-01,505,00:00:00,Friday,dundas west stationt to broadview station,General Delay,7.0,14.0,w,4028,2016-01-01 00:00:00,"[0.0, 0.0]",0.000000,0.000000
1,2016-01-01,511,02:14:00,Friday,fleet st. and strachan,Mechanical,10.0,20.0,e,4018,2016-01-01 02:14:00,"[43.6362976, -79.4096351]",43.636298,-79.409635
2,2016-01-01,301,02:22:00,Friday,queen st. west and roncesvalles,Mechanical,9.0,18.0,w,4201,2016-01-01 02:22:00,"[43.64533489999999, -79.4131843]",43.645335,-79.413184
3,2016-01-01,301,03:28:00,Friday,lake shore blvd. and superior st.,Mechanical,20.0,40.0,e,4251,2016-01-01 03:28:00,"[43.61496169999999, -79.4886581]",43.614962,-79.488658
4,2016-01-01,501,14:28:00,Friday,roncesvalles to neville park,Mechanical,6.0,12.0,e,4242,2016-01-01 14:28:00,"[0.0, 0.0]",0.000000,0.000000


In [ ]:
dfn.shape

(69603, 14)

In [ ]:
file_outname = "2014_2018_df_cleaned_keep_bad_loc_geocoded_apr29.csv"
dfn.to_csv(path+file_outname)

# 정리된 데이터의 시각화

In [17]:
!pip install pixiedust

     |████████████████████████████████| 197 kB 2.6 MB/s 
  Created wheel for pixiedust: filename=pixiedust-1.1.19-py3-none-any.whl size=321804 sha256=932d58d1c96aa8ac65376566dd2ec22186ab9cfb673bc8c11c256ef6d3169c3d
  Stored in directory: /root/.cache/pip/wheels/05/07/e7/8aca0e820027a63157a916424fd748fb2a2a3e71de5e08eeb8
Successfully built pixiedust


In [18]:
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [ ]:
file_outname = "2014_2018_df_cleaned_keep_bad_loc_geocoded_apr29.csv"
df = pd.read_csv(path+'/'+file_outname)

In [ ]:
display(df)